In [7]:
import sqlite3
import contextlib
from enum import Enum
from fractions import Fraction
import random
import numpy as np
from scipy import stats
import multiprocessing as mp
import pickle
from functools import reduce

## Database

In [9]:
def dict_factory(c, row):
    return {
        key: val 
        for key, val in zip(
            [col[0] for col in c.description],
            row
    )}

def get_db_solos():
    with sqlite3.connect('data/wjazzd.db') as db:
        db.row_factory = dict_factory
        with contextlib.closing(db.cursor()) as c:
            solo_q = '''
            SELECT 
                si.melid AS mid,
                ci.form AS form,
                si.key AS key
            FROM composition_info AS ci
            INNER JOIN solo_info AS si
            ON ci.compid = si.compid
            WHERE
                ci.tonalitytype = 'MODAL' AND
                si.instrument IN ('ts', 'as', 'ss', 'tp')
            ORDER BY
                ci.compid ASC,
                si.melid ASC
            '''
            c.execute(solo_q)
            solos = c.fetchall()

            for solo in solos:
                phrases_q = f'''
                SELECT start, end
                FROM sections
                WHERE type = 'PHRASE' AND melid = {solo['mid']}
                ORDER BY value
                '''
                c.execute(phrases_q)
                phrases = c.fetchall()

                solo['phrases'] = []
                for phrase in phrases:
                    notes_q = f'''
                    SELECT
                        s.value AS chord,
                        m.pitch AS pitch,
                        m.duration AS duration,
                        m.beatdur AS beatdur,
                        m.period AS period,
                        m.division AS division,
                        m.bar AS bar,
                        m.beat AS beat,
                        m.tatum AS tatum
                    FROM melody AS m
                    LEFT JOIN (
                        SELECT MIN(eventid) AS meventid
                        FROM melody
                        WHERE melid = {solo['mid']}
                    )
                    INNER JOIN sections AS s
                    ON 
                        m.melid = s.melid AND
                        m.eventid - meventid >= s.start AND
                        m.eventid - meventid <= s.end
                    WHERE
                        m.melid = {solo['mid']} AND
                        m.eventid - meventid >= {phrase['start']} AND
                        m.eventid - meventid <= {phrase['end']} AND
                        s.type = 'CHORD'
                    ORDER BY
                        m.eventid ASC
                    '''
                    c.execute(notes_q)
                    solo['phrases'].append(c.fetchall())

            return solos

## Utils

In [10]:
def note_offset(note):
    match note[0]:
        case 'C':
            offset = 0
        case 'D':
            offset = 2
        case 'E':
            offset = 4
        case 'F':
            offset = 5
        case 'G':
            offset = 7
        case 'A':
            offset = 9
        case 'B':
            offset = 11
    
    if len(note) > 1 and note[1] == 'b':
        return (offset - 1, note[2:])
    elif len(note) > 1 and note[1] == '#':
        return (offset + 1, note[2:])
    else:
        return (offset, note[1:])

In [11]:
# https://jazzomat.hfm-weimar.de/melospy/annotations.html#chords

def chord_st(chord):
    stv = np.array([0, 4, 7]) # 1 M3 P5

    if len(chord) >= 1:
        if chord[0] == 'm' or chord[0] == '-':
            stv[1] -= 1 # m3
        elif chord[0] == '+':
            stv[2] += 1 # A5
        elif chord[0] == 'o':
            stv[1] -= 1 # m3
            stv[2] -= 1 # d5
        elif chord.startswith('sus'):
            stv[1] += 1 # P4

    if 'j7' in chord:
        stv = np.append(stv, 11) # M7
    elif '7' in chord:
        stv = np.append(stv, 10) # m7

    return stv

def color_st(chord):
    stv = np.empty(0)

    if '6' in chord:
        stv = np.append(stv, 9) # M6

    if '9#' in chord:
        stv = np.append(stv, 3) # A9
    elif '9b' in chord:
        stv = np.append(stv, 1) # m9
    elif '9' in chord:
        stv = np.append(stv, 2) # M9


    if '11#' in chord:
        stv = np.append(stv, 6) # A11
    elif '11' in chord:
        stv = np.append(stv, 5) # P11

    return stv


def helpful_st(chord):
    chord_stv = chord_st(chord)
    color_stv = color_st(chord)

    return np.union1d(chord_stv, color_stv)

In [12]:
def scale_mode(key):
    match key:
        case 'maj':
            return np.array([0, 2, 4, 5, 7, 9, 11])
        case 'min':
            return np.array([0, 2, 3, 5, 7, 8, 10])
        case 'ion':
            return np.array([0, 2, 4, 5, 7, 9, 11])
        case 'dor':
            return np.array([0, 2, 3, 5, 7, 9, 10])
        case 'phr':
            return np.array([0, 1, 3, 5, 7, 8, 10])
        case 'lyd':
            return np.array([0, 2, 4, 6, 7, 9, 11])
        case 'mix':
            return np.array([0, 2, 4, 5, 7, 9, 10])
        case 'aeol':
            return np.array([0, 2, 3, 5, 7, 8, 10])
        case 'lok':
            return np.array([0, 1, 3, 5, 6, 8, 10])
        case 'chrom':
            return np.arange(0, 12, 1)
        case _:
            return np.empty(0)

## Models

In [13]:
class NoteTone(Enum):
    CHORD = 1
    COLOR = 2
    HELPFUL = 3
    SCALE = 4
    ARBITRARY = 5
    REST = 6

    def from_pitch(pitch, chord, scale):
        if chord != 'NC': # no chord
            chord_offs, gen_chord = note_offset(chord)
            rel_chord_st = (pitch - chord_offs) % 12 # semitones above root chord note

            if rel_chord_st in chord_st(gen_chord):
                return NoteTone.CHORD
            elif rel_chord_st in color_st(gen_chord):
                return NoteTone.COLOR
            
        if scale != '':
            scale_offs, gen_scale = note_offset(scale)
            rel_scale_st = (pitch - scale_offs) % 12 # semitones above scale root note
            
            if rel_scale_st in scale_mode(gen_scale[1:]):
                return NoteTone.SCALE

        return NoteTone.ARBITRARY
    

In [14]:
# https://jazzomat.hfm-weimar.de/melospy/metrical_system.html

class NoteDuration(Enum):
    WHOLE = Fraction(4)
    HALF = Fraction(2)
    QUARTER = Fraction(1)
    QUARTER3 = Fraction(2, 3)
    EIGHTH = Fraction(1, 2)
    EIGHTH3 = Fraction(1, 3)
    SIXTEENTH = Fraction(1, 4)
    THIRTYSECOND = Fraction(1, 8)
    SIXTYFOURTH = Fraction(1, 16)

    def from_metric(duration, beatdur):
        rel_dur = duration / beatdur
        
        if rel_dur <= NoteDuration.SIXTYFOURTH.value:
            return [NoteDuration.SIXTYFOURTH]
        elif rel_dur <= NoteDuration.THIRTYSECOND.value:
            return [NoteDuration.THIRTYSECOND]
        elif rel_dur <= NoteDuration.SIXTEENTH.value:
            return [NoteDuration.SIXTEENTH]
        elif rel_dur <= NoteDuration.EIGHTH3.value:
            return [NoteDuration.EIGHTH3]
        elif rel_dur <= NoteDuration.EIGHTH.value:
            return [NoteDuration.EIGHTH]
        elif rel_dur <= NoteDuration.QUARTER3.value:
            return [NoteDuration.QUARTER3]
        else:
            n_whole = rel_dur // 4
            n_half = (rel_dur - 4 * n_whole) // 2
            n_quarter = rel_dur - 4 * n_whole - 2 * n_half
            return [NoteDuration.WHOLE] * int(n_whole) + \
                   [NoteDuration.HALF] * int(n_half) + \
                   [NoteDuration.QUARTER] * int(n_quarter)
        
    def fill_beats(duration):
        durs = []
        for dur in NoteDuration:
            n_dur = duration // dur.value
            durs += [dur] * n_dur
            duration -= n_dur * dur.value

        return durs

        
    def get_rest(currnote, currdurs, nextnote):
        dur_beats = reduce(lambda acc, d: acc + d.value, currdurs, Fraction(0))
        bar_beats = (nextnote['bar'] - currnote['bar']) * currnote['period']
        beats = nextnote['beat'] - currnote['beat']
        tatums = Fraction(nextnote['tatum'], nextnote['division']) - \
                 Fraction(currnote['tatum'], currnote['division'])

        beats_diff = bar_beats + beats + tatums
        if beats_diff <= 0:
            raise ValueError('Notes in wrong order')
        rest_beats = beats_diff - dur_beats + (NoteDuration.SIXTYFOURTH.value / 2) # allow overestimate rest
        if rest_beats <= 0:
            return []
        
        return NoteDuration.fill_beats(rest_beats)

In [15]:
class Note:
    def __init__(self, tone, duration):
        self.tone = tone
        self.duration = duration

    def to_tuple(self):
        return (self.tone, self.duration)

class Phrase:
    def __init__(self, notes, is_first=False):
        self.notes = notes
        self.next = None
        self.is_first = is_first
        self.similar_next = []
        self.similar_next_end = []

    @property
    def length(self):
        n, d = reduce(lambda acc, n: acc + n.duration.value, self.notes, Fraction(0)).as_integer_ratio()
        return n / d

    def get_beat(self, idx):
        res_notes = []
        beat_c = 0
        filling = False

        for note in self.notes:
            if filling:
                if beat_c < idx + 1:
                    to_add_dur = min(note.duration.value, idx + 1 - beat_c)
                    durs = NoteDuration.fill_beats(to_add_dur)
                    res_notes += list(map(lambda d: Note(note.tone, d), durs))
                else:
                    return Phrase(res_notes)
                beat_c += note.duration.value
            else:
                beat_cn = beat_c + note.duration.value
                if beat_cn >= idx:
                    to_add_dur = min(beat_cn - idx, 1)
                    filling = True
                    durs = NoteDuration.fill_beats(to_add_dur)
                    res_notes += list(map(lambda d: Note(note.tone, d), durs))
                beat_c = beat_cn

        rest_to_add = min(idx + 1 - beat_c, 1)
        if rest_to_add > 0:
            durs = NoteDuration.fill_beats(rest_to_add)
            res_notes += list(map(lambda d: Note(NoteTone.REST, d), durs))

        return Phrase(res_notes)
    
    def get_tone_duration(self, tone):
        tone_f = filter(lambda n: n.tone == tone, self.notes)
        return Phrase(tone_f).length
    
    def to_list(self):
        return list(map(lambda n: n.to_tuple(), self.notes))

class PhraseSeqSimilar:
    def __init__(self):
        self.all = []
        self.start = []
        self.end = []

    def add_phrase(self, idx, phrase):
        self.all.append(phrase)
        if phrase.is_first:
            self.start.append(idx)
        if phrase.next == None:
            self.end.append(idx)

    def to_list(self):
        return list(map(lambda ph: ph.to_list(), self.all))


## Preprocessing

In [16]:
def preprocess(solos):
    phrases = PhraseSeqSimilar()
    ph_idx = 0

    for solo in solos:
        ph = Phrase([], is_first=True)
        for phrase in solo['phrases']:
            for i, curr in enumerate(phrase):
                tone = NoteTone.from_pitch(curr['pitch'], curr['chord'], solo['key'])
                durs = NoteDuration.from_metric(curr['duration'], curr['beatdur'])

                for dur in durs:
                    ph.notes.append(Note(tone, dur))

                if i < len(phrase) - 1:
                    next = phrase[i + 1]
                    rests = NoteDuration.get_rest(curr, durs, next)

                    for dur in rests:
                        ph.notes.append(Note(NoteTone.REST, dur))

            ph.next = ph_idx + 1
            phrases.add_phrase(ph_idx, ph)
            ph = Phrase([])
            ph_idx += 1

        last_ph = phrases.all[-1]
        last_ph.next = -1
        phrases.end.append(ph_idx - 1)

    return phrases

## Similar next phrases

In [17]:
# difference between phrases heuristic
def phrase_diff(phrase1, phrase2):
    discount = 0.8
    weight = 1

    diff = 0
    max_len = np.ceil(max(phrase2.length, phrase1.length))
    
    for i in range(int(max_len)):
        phrase1_b = phrase1.get_beat(i)
        phrase2_b = phrase2.get_beat(i)
        phrase1_bn = phrase1_b.length # actual lengths
        phrase2_bn = phrase2_b.length

        p1_cn = phrase1_b.get_tone_duration(NoteTone.CHORD) / phrase1_bn
        p2_cn = phrase2_b.get_tone_duration(NoteTone.CHORD) / phrase2_bn
        p1_ln = phrase1_b.get_tone_duration(NoteTone.COLOR) / phrase1_bn
        p2_ln = phrase2_b.get_tone_duration(NoteTone.COLOR) / phrase2_bn
        p1_sn = phrase1_b.get_tone_duration(NoteTone.SCALE) / phrase1_bn
        p2_sn = phrase2_b.get_tone_duration(NoteTone.SCALE) / phrase2_bn
        p1_an = phrase1_b.get_tone_duration(NoteTone.ARBITRARY) / phrase1_bn
        p2_an = phrase2_b.get_tone_duration(NoteTone.ARBITRARY) / phrase2_bn
        p1_rn = phrase1_b.get_tone_duration(NoteTone.REST) / phrase1_bn
        p2_rn = phrase2_b.get_tone_duration(NoteTone.REST) / phrase2_bn

        diff += weight * np.abs(p2_cn - p1_cn) / 2
        diff += weight * np.abs(p2_ln - p1_ln) / 2
        diff += weight * np.abs(p2_cn + p2_ln - p1_cn - p1_ln) / 2
        diff += weight * np.abs(p2_sn - p1_sn)
        diff += weight * np.abs(p2_an - p1_an)
        diff += weight * np.abs(p2_rn - p1_rn)

        weight *= discount

    return diff

In [18]:
def get_similar_next(phrase_idx, phrases, limit=None):
    ph_next_n = phrases.all[phrase_idx].next
    if ph_next_n == -1:
        return ([], [])

    ph_next = phrases.all[ph_next_n]
    all_ph_s = list(filter(lambda ph_n: phrases.all[ph_n].next != -1, range(len(phrases.all))))
    end_ph_s = range(len(phrases.end))

    if not limit:
        all_k = len(all_ph_s)
        end_k = len(end_ph_s)
    else:
        all_k = min(len(all_ph_s), limit)
        end_k = min(len(end_ph_s), limit)

    if ph_next.next == -1:
        all_ph_s = random.sample(all_ph_s, k=all_k)

        end_f = list(filter(lambda ph_n: ph_n != ph_next_n, end_ph_s))
        end_ph_s = random.sample(end_f, k=end_k - 1)
        end_ph_s.append(ph_next_n)
    else:
        all_f = list(filter(lambda ph_n: ph_n != ph_next_n, all_ph_s))
        all_ph_s = random.sample(all_f, k=all_k - 1)
        all_ph_s.append(ph_next_n)
        
        end_ph_s = random.sample(end_ph_s, k=end_k)

    ph_key = lambda ph_n: phrase_diff(phrases.all[ph_n], ph_next)

    return (
        sorted(all_ph_s, key=ph_key),
        sorted(end_ph_s, key=ph_key)
    )

def get_similar_next_batch(phrase_n_batch, phrases, limit=None):
    return {
        ph_idx: get_similar_next(ph_idx, phrases, limit)
        for ph_idx in phrase_n_batch
    }
        

## Learning module

In [19]:
def learn_phrases(solos, out_file=None, max_workers=10, limit_similar=None):
    phrases = preprocess(solos)

    next_results = {}
    def store_results(res):
        next_results.update(res)

    phrases_n = range(len(phrases.all))
    with mp.Pool(processes=max_workers) as pool:
        results = [
            pool.apply_async(
                get_similar_next_batch,
                args=(phrase_batch, phrases, limit_similar),
                callback=store_results,
                error_callback=print
            )
            for phrase_batch in np.array_split(phrases_n, max_workers)
        ]

        for r in results:
            r.wait()

    for ph_idx, (ph_next, ph_next_end) in next_results.items():
        phrase = phrases.all[ph_idx]
        phrase.similar_next = ph_next
        phrase.similar_next_end = ph_next_end

    if out_file:
        with open(out_file, 'wb') as fh:
            pickle.dump(phrases, fh)

    return phrases

In [20]:
solos = get_db_solos()
learn_phrases(solos, './data/modal_phrases_all.pickle', max_workers=mp.cpu_count())

In [21]:
def load_phrases(in_file):
    with open(in_file, 'rb') as fh:
        return pickle.load(fh)

In [15]:
phrases = load_phrases('./data/modal_phrases_all.pickle')

## Sampling

In [16]:
def sample_melody(phrases, min_beats, max_rest, phrase_mutation_prob):
    fallthrough_dist = stats.geom(1 - phrase_mutation_prob)

    ph_start_idx = random.choice(phrases.start)
    ph_start = phrases.all[ph_start_idx]

    ph_prev = ph_start
    melody = [ph_start.notes.copy()]
    min_beats -= max_rest
    min_beats -= ph_start.length

    while min_beats > 0:
        ph_next_sim_idx = (fallthrough_dist.rvs() - 1) % len(ph_prev.similar_next)
        ph_next_idx = ph_prev.similar_next[ph_next_sim_idx]
        ph_next = phrases.all[ph_next_idx]
        
        ph_prev = ph_next
        melody.append(ph_next.notes.copy())

        min_beats -= ph_next.length

    ph_last_sim_idx = (fallthrough_dist.rvs() - 1) % len(ph_prev.similar_next_end)
    ph_last_idx = ph_prev.similar_next_end[ph_last_sim_idx]
    ph_last = phrases.all[ph_last_idx]

    melody.append(ph_last.notes.copy())

    rest_points = len(melody) - 1
    rest_split = np.array(sorted([
        random.randrange(0, max_rest + 1, 1)
        for _ in range(rest_points)
    ]))

    rest_split[1:] = rest_split[1:] - rest_split[:-1]

    melody_res = list(sum([
        (
            notes,
            list(map(
                lambda d: Note(NoteTone.REST, d),
                NoteDuration.fill_beats(rest_dur)
            ))
        )
        for notes, rest_dur in zip(melody, rest_split)
    ], ()))
    melody_res.append(melody[-1])

    return [n for ns in melody_res for n in ns]

## Note selection

In [17]:
def select_pitch(tone, prev_tone, chord, scale, tone_mutation_prob, pitch_range, std):
    if tone == NoteTone.REST:
        return -1

    elif random.uniform(0, 1) <= tone_mutation_prob:
        match tone:
            case NoteTone.CHORD:
                return select_pitch(NoteTone.HELPFUL, prev_tone, chord, scale, 0, pitch_range, std)
            case NoteTone.COLOR:
                return select_pitch(NoteTone.HELPFUL, prev_tone, chord, scale, 0, pitch_range, std)
            case NoteTone.HELPFUL:
                return select_pitch(NoteTone.ARBITRARY, prev_tone, chord, scale, 0, pitch_range, std)
            case NoteTone.SCALE:
                return select_pitch(NoteTone.ARBITRARY, prev_tone, chord, scale, 0, pitch_range, std)
            case NoteTone.ARBITRARY:
                return select_pitch(NoteTone.HELPFUL, prev_tone, chord, scale, 0, pitch_range, std)

    stv = np.empty(0)
    if tone == NoteTone.CHORD:
        if chord != 'NC':
            chord_offs, gen_chord = note_offset(chord)
            stv = chord_st(gen_chord) + chord_offs
    elif tone == NoteTone.COLOR:
        if chord != 'NC':
            chord_offs, gen_chord = note_offset(chord)
            stv = color_st(gen_chord) + chord_offs
        if len(stv) == 0:
            stv = chord_st(gen_chord) + chord_offs
    elif tone == NoteTone.HELPFUL:
        if chord != 'NC':
            chord_offs, gen_chord = note_offset(chord)
            stv = helpful_st(gen_chord) + chord_offs
    elif tone == NoteTone.SCALE:
        if scale != '':
            scale_offs, gen_scale = note_offset(scale)
            stv = helpful_st(gen_scale) + scale_offs
    elif tone == NoteTone.ARBITRARY:
        if chord != 'NC':
            chord_offs, gen_chord = note_offset(chord)
            chord_stv = helpful_st(gen_chord) + chord_offs
        else:
            chord_stv = np.empty(0)

        if scale != '':
            scale_offs, gen_scale = note_offset(scale)
            scale_stv = helpful_st(gen_scale) + scale_offs
        else:
            scale_stv = np.empty(0)

        stv = np.setdiff1d(
            np.arange(0, 12),
            np.union1d(chord_stv, scale_stv)
        )
    if len(stv) == 0:
        stv = np.arange(0, 12)


    prev_pitch = prev_tone
    next_pitch = stats.norm(prev_pitch, std).rvs()
    next_octave = next_pitch // 12

    min_pitch, max_pitch = pitch_range
    stv_padded = np.append(stv, (stv[-1] - 12, stv[0] + 12))
    stv_allowed = []
    for st in stv_padded:
        nst = st + next_octave * 12
        if nst > max_pitch:
            rm_octave = (nst - max_pitch) // 12 + 1
            nst -= rm_octave * 12
        elif nst < min_pitch:
            add_octave = (min_pitch - nst) // 12 + 1
            nst += add_octave * 12

        stv_allowed.append(nst)
        
    return sorted(stv_allowed, key=lambda st: np.abs(next_pitch - st))[0]

In [18]:
def select_notes(abs_melody, chords, scale, init_pitch, tone_mutation_prob, pitch_range, std):
    melody = []
    chord, dur = chords.pop(0)
    prev_pitch = init_pitch

    for note in abs_melody:
        pitch = select_pitch(note.tone, prev_pitch, chord, scale, tone_mutation_prob, pitch_range, std)
        prev_pitch = pitch
        melody.append(Note(pitch, note.duration))

        dur -= note.duration.value
        if dur <= 0:
            try:
                chord, next_dur = chords.pop(0)
                dur = next_dur + dur
            except IndexError:
                dur = 1000

    return melody

## Improvising module

In [19]:
def improvise_melody(phrases, chords, scale, out_file=None, phrase_mutation_prob=0.5, init_pitch=75, tone_mutation_prob=0.05, pitch_range=(60, 108), std=5):
    beats = reduce(lambda acc, c: acc + c[1], chords, 0)
    max_rest = beats // 5
    min_beats = beats - max_rest

    abs_melody = sample_melody(phrases, min_beats, max_rest, phrase_mutation_prob)
    melody = select_notes(abs_melody, chords, scale, init_pitch, tone_mutation_prob, pitch_range, std)

    if out_file:
        with open(out_file, 'wb') as fh:
            pickle.dump(melody, fh)

    return melody

In [20]:
def load_melody(in_file):
    with open(in_file, 'rb') as fh:
        return pickle.load(fh)

In [21]:
m = improvise_melody(phrases, [('Cj7', 12), ('Eb-911', 4), ('F#6', 4), ('F#7', 8), ('E+711#', 12), ('Dj79', 8), ('F', 8), ('G76', 4)], 'C-ion', './data/improv/test.pickle')
list(map(lambda n: n.to_tuple(), m))

[(72.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (63.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (63.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (-1, <NoteDuration.EIGHTH: Fraction(1, 2)>),
 (63.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (-1, <NoteDuration.QUARTER3: Fraction(2, 3)>),
 (-1, <NoteDuration.SIXTYFOURTH: Fraction(1, 16)>),
 (61.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (-1, <NoteDuration.QUARTER3: Fraction(2, 3)>),
 (-1, <NoteDuration.SIXTYFOURTH: Fraction(1, 16)>),
 (63.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (67.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (-1, <NoteDuration.EIGHTH: Fraction(1, 2)>),
 (63.0, <NoteDuration.THIRTYSECOND: Fraction(1, 8)>),
 (-1, <NoteDuration.QUARTER3: Fraction(2, 3)>),
 (-1, <NoteDuration.THIRTYSECOND: Fraction(1, 8)>),
 (-1, <NoteDuration.SIXTYFOURTH: Fraction(1, 16)>),
 (63.0, <NoteDuration.THIRTYSECOND: Fraction(1, 8)>),
 (-1, <NoteDuration.QUARTER3: Fraction(2, 3)>),
 (-1, <NoteDuration.THIRTYSECOND: Fraction(

In [22]:
m_ = load_phrases('./data/improv/test.pickle')
list(map(lambda n: n.to_tuple(), m_))

[(72.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (63.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (63.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (-1, <NoteDuration.EIGHTH: Fraction(1, 2)>),
 (63.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (-1, <NoteDuration.QUARTER3: Fraction(2, 3)>),
 (-1, <NoteDuration.SIXTYFOURTH: Fraction(1, 16)>),
 (61.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (-1, <NoteDuration.QUARTER3: Fraction(2, 3)>),
 (-1, <NoteDuration.SIXTYFOURTH: Fraction(1, 16)>),
 (63.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (67.0, <NoteDuration.SIXTEENTH: Fraction(1, 4)>),
 (-1, <NoteDuration.EIGHTH: Fraction(1, 2)>),
 (63.0, <NoteDuration.THIRTYSECOND: Fraction(1, 8)>),
 (-1, <NoteDuration.QUARTER3: Fraction(2, 3)>),
 (-1, <NoteDuration.THIRTYSECOND: Fraction(1, 8)>),
 (-1, <NoteDuration.SIXTYFOURTH: Fraction(1, 16)>),
 (63.0, <NoteDuration.THIRTYSECOND: Fraction(1, 8)>),
 (-1, <NoteDuration.QUARTER3: Fraction(2, 3)>),
 (-1, <NoteDuration.THIRTYSECOND: Fraction(